# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,8.23,85,98,4.02,NZ,1691607960
1,1,west island,-12.1568,96.8225,24.99,83,75,5.66,CC,1691607960
2,2,iqaluit,63.7506,-68.5145,4.85,100,100,3.58,CA,1691607722
3,3,sarakhs,36.5449,61.1577,30.09,23,9,2.50,IR,1691607961
4,4,hermanus,-34.4187,19.2345,14.33,88,0,4.93,ZA,1691607961


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp = city_data_df.loc[city_data_df["Max Temp"] < 10]

# Drop any rows with null values
clean_df = ideal_temp.dropna(how = "any")

# Display sample data
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,8.23,85,98,4.02,NZ,1691607960
2,2,iqaluit,63.7506,-68.5145,4.85,100,100,3.58,CA,1691607722
11,11,papatowai,-46.5619,169.4708,1.83,86,91,2.54,NZ,1691607963
14,14,port-aux-francais,-49.3500,70.2167,2.48,78,87,12.06,TF,1691607964
21,21,severo-yeniseyskiy,60.3725,93.0408,9.26,92,2,2.42,RU,1691607965


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df[["Country","City","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,Country,City,Lat,Lng,Humidity,Hotel Name
0,NZ,waitangi,-43.9535,-176.5597,85,
2,CA,iqaluit,63.7506,-68.5145,100,
11,NZ,papatowai,-46.5619,169.4708,86,
14,TF,port-aux-francais,-49.3500,70.2167,78,
21,RU,severo-yeniseyskiy,60.3725,93.0408,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10_000
params = {"categories":"accommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waitangi - nearest hotel: Hotel Chathams
iqaluit - nearest hotel: Frobisher Inn
papatowai - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
severo-yeniseyskiy - nearest hotel: Актолик
puerto natales - nearest hotel: Dorotea Patagonia Hostel
grytviken - nearest hotel: No hotel found
vorgashor - nearest hotel: София
invercargill - nearest hotel: The Grand
aasiaat - nearest hotel: SØMA
akureyri - nearest hotel: Hótel Norðurland
punta arenas - nearest hotel: Hotel Lacolet
qaqortoq - nearest hotel: Hotel Qaqortoq
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
nagqu - nearest hotel: Naqu Hotel
olonkinbyen - nearest hotel: No hotel found
labytnangi - nearest hotel: Семь лиственниц
port augusta - nearest hotel: Hotel Commonwealth
bilibino - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
el calafate - nearest hotel: Hostería Las Avutardas
masterton - nearest hotel: Copthorne Hotel Solway Park
nuuk - nearest

,Country,City,Lat,Lng,Humidity,Hotel Name
0,NZ,waitangi,-43.9535,-176.5597,85,Hotel Chathams
2,CA,iqaluit,63.7506,-68.5145,100,Frobisher Inn
11,NZ,papatowai,-46.5619,169.4708,86,No hotel found
14,TF,port-aux-francais,-49.3500,70.2167,78,Keravel
21,RU,severo-yeniseyskiy,60.3725,93.0408,92,Актолик
24,CL,puerto natales,-51.7236,-72.4875,88,Dorotea Patagonia Hostel
31,GS,grytviken,-54.2811,-36.5092,66,No hotel found
39,RU,vorgashor,67.5833,63.9500,75,София
42,NZ,invercargill,-46.4000,168.3500,94,The Grand
49,GL,aasiaat,68.7098,-52.8699,75,SØMA


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)